In [ ]:
tf.__version__

In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [ ]:
from google.colab import files
src = list(files.upload().values())

In [ ]:
import os
import tensorflow as tf
import keras
from keras.applications import Xception
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import numpy as np
import PIL

In [ ]:
# 체크포인트 경로
checkpoint_path = "/content/MyDrive/My Drive/Colab Notebooks/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# 체크포인트 불러오기
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/MyDrive/My Drive/Colab Notebooks/cp-0028.ckpt


In [ ]:
model=keras.models.load_model("/content/MyDrive/My Drive/Colab Notebooks/faceModel.h5")
model.summary()

In [ ]:
inputs = Input(shape=(224, 224, 3))
model= Xception(include_top=False,weights=None,input_tensor=inputs,input_shape=(224,224,3),pooling="avg")
x = model.output
x = Dense(1024, name='fully', kernel_initializer='random_uniform')(x)
x = Activation('relu')(x)
x = Dense(512, kernel_initializer='random_uniform')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
prediction = Dense(units=3, kernel_initializer="he_normal", use_bias=False, activation="softmax",name="pred_age")(x)
model = Model(inputs=model.input, outputs=prediction)
model.summary()

In [ ]:
model.load_weights(latest)

In [ ]:
# 훈련 데이터 목록 생성
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '/content/MyDrive/My Drive/Colab Notebooks/dataset/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

# 검증 데이터 목록 생성
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        '/content/MyDrive/My Drive/Colab Notebooks/dataset/vaild',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 22691 images belonging to 3 classes.
Found 7475 images belonging to 3 classes.


In [ ]:
#sgd 시험
sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# 컴파일
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])

In [ ]:
# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, period=1)  # verbose: 진행상황 출력 여부
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10)

In [ ]:
# 훈련 및 검증
history = model.fit_generator(train_generator,
                              epochs=50,
                              validation_data=val_generator,
                              callbacks=[cp_callback])


In [ ]:
print("-- evaluate --")
scores = model.evaluate_generator(val_generator)
print(val_generator.class_indices)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# 테스트 데이터 목록 생성
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/content/MyDrive/My Drive/Colab Notebooks/dataset/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 7542 images belonging to 3 classes.


In [ ]:
print("-- Predict --")
scores = model.evaluate_generator(test_generator)
print(test_generator.class_indices)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Predict --
{'middle': 0, 'old': 1, 'young': 2}
acc: 91.59%


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(history.history['acc'],'r')
plt.plot(history.history['val_acc'],'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.show()

In [ ]:
model.save("/content/MyDrive/My Drive/Colab Notebooks/faceModel.h5")

In [ ]:
model_json = model.to_json()
with open('/content/MyDrive/My Drive/Colab Notebooks/keras_model.json', "w") as json_file:
	json_file.write(model_json)

In [ ]:
from tensorflow import keras
model = keras.models.load_model("/content/MyDrive/My Drive/Colab Notebooks/faceModel.h5", compile=False)

#export_path = "/content/MyDrive/My Drive/Colab Notebooks/faceModel"'
#model.save(export_path, save_format='tf')

In [ ]:
print(' ----FREEZE GRAPH----- ')
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.python.tools import optimize_for_inference_lib
#loaded = tf.keras.models.load_model("/content/MyDrive/My Drive/Colab Notebooks/faceModel.h5", compile=False)
flatten_input=tf.TensorSpec(shape=(None,224, 224, 3), dtype=tf.float32)
f = tf.function(model).get_concrete_function(flatten_input)
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()
with tf.io.gfile.GFile('/content/MyDrive/My Drive/Colab Notebooks/frozen_graph.pb', 'wb') as f:
  f.write(graph_def.SerializeToString())
print('done')

 ----FREEZE GRAPH----- 
done
